# 空氣污染監測網 網路爬蟲實作練習


* 能夠利用 selenium + BeautifulSoup 撰寫爬蟲，並存放到合適的資料結構


## 作業目標

根據範例 ，完成以下問題：

* ① 取出 台北市士林區 2018/01 – 2018/08 的 SO2 資料
* ② 取出 台北市士林區 2018/01 – 2018/08 的 SO2、CO 資料





### ① 取出 台北市士林區 2018/01 – 2018/08 的 SO2 資料

In [1]:
# 打開瀏覽器
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
browser = webdriver.Chrome(executable_path='chromedriver')
browser.get("http://taqm.epa.gov.tw/taqm/tw/MonthlyAverage.aspx")

# 模擬使用者操作行為，選擇/點擊
selectSite = Select(browser.find_element_by_id("ctl05_ddlSite"))
selectSite.select_by_value('11')
selectYear = Select(browser.find_element_by_id("ctl05_ddlYear"))
selectYear.select_by_value('2018')
browser.find_element_by_id('ctl05_btnQuery').click()

# 取得資料
import time
time.sleep(10) # 10秒後會關掉瀏覽器

html_source = browser.page_source
soup = BeautifulSoup(html_source, "html5lib")
table = soup.find('table', class_='TABLE_G')

# 關閉瀏覽器
browser.quit()

data = {}
key = ''
date = ''
items = table.find_all('td')
for item in items:
    if 'style' in item.attrs:
        key = item.text
        data[key]={}
    elif 'class' not in item.attrs:
        if item.text.strip() != '':
            if '/' in item.text:
                date = item.text
            else:
                data[key][date] = item.text
# print(data) # 取出所有資料

import pandas as pd
all_data = pd.DataFrame(data)[:-1]
all_data = pd.DataFrame(data)[:-1]
print(all_data[:8]['SO2'])

WebDriverException: Message: 'chromedriver' executable needs to be in PATH. Please see https://sites.google.com/a/chromium.org/chromedriver/home


### ② 取出 台北市士林區 2018/01 – 2018/08 的 SO2、CO 資料

In [2]:
print(all_data[:8][['SO2', 'CO']]) # 補上CO

NameError: name 'all_data' is not defined

In [3]:
'''
官方寫法(迴圈開始)
'''
# 打開瀏覽器
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
browser = webdriver.Chrome(executable_path='chromedriver')
browser.get("http://taqm.epa.gov.tw/taqm/tw/MonthlyAverage.aspx")

# 模擬使用者操作行為，選擇/點擊
selectSite = Select(browser.find_element_by_id("ctl05_ddlSite"))
selectSite.select_by_value('11')
selectYear = Select(browser.find_element_by_id("ctl05_ddlYear"))
selectYear.select_by_value('2018')
browser.find_element_by_id('ctl05_btnQuery').click()

# 取得資料
import time
time.sleep(10) # 10秒後會關掉瀏覽器

html_source = browser.page_source
soup = BeautifulSoup(html_source, "html5lib")
table = soup.find('table', class_='TABLE_G')

# 關閉瀏覽器
browser.quit()

d = {}
for tr in table.find_all('tr')[1:]: # 跳過column欄位項目

    for i, td in enumerate(tr.find_all('td')):
        
        if len(tr.find_all('td')) == 5: # 有5個td(欄目) + 第一個月資料
            if i == 0: # 第一個td是監測項目
                obs = td.text
                d.setdefault(obs, {}) # {'項目名稱': {}}
            if i == 2: # 第一個月監測日期
                date = td.text
            if i == 3: # 第一個監測值
                value = td.text
                d[obs][date] = value  # 字典裡面還是字典，進行新增資料
        
        if len(tr.find_all('td')) == 3: # 有3個td為第二個月之後資料
            if i == 0: # 第二個月監測日期
                date = td.text
            if i == 1: # 第二個監測值
                value = td.text
                d[obs][date] = value
print(d)

WebDriverException: Message: 'chromedriver' executable needs to be in PATH. Please see https://sites.google.com/a/chromium.org/chromedriver/home


In [4]:
# 整理成 DataFrame
import pandas as pd

d_SO2 = [{'時間': key, 'SO2': value} for key, value in d['SO2'].items() if key < '2018/09' ]
d_CO = [{'時間': key, 'CO': value} for key, value in d['CO'].items() if key < '2018/09' ]

pd.merge(pd.DataFrame(d_SO2), pd.DataFrame(d_CO))

NameError: name 'd' is not defined

In [ ]:
其他寫法:

# 打開瀏覽器
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
browser = webdriver.Chrome(executable_path='chromedriver')
browser.get("http://taqm.epa.gov.tw/taqm/tw/MonthlyAverage.aspx")

# 模擬使用者操作行為，選擇/點擊
selectSite = Select(browser.find_element_by_id("ctl05_ddlSite"))
selectSite.select_by_value('11')
selectYear = Select(browser.find_element_by_id("ctl05_ddlYear"))
selectYear.select_by_value('2018')
browser.find_element_by_id('ctl05_btnQuery').click()

# 取得資料
import time
time.sleep(15) # 15秒後會關掉瀏覽器

html_source = browser.page_source
soup = BeautifulSoup(html_source, "html5lib")
table = soup.find('table', class_='TABLE_G')

# 關閉瀏覽器
browser.quit()

# 創建dictionary
data = {}
for tr in table.find_all('tr')[1:]: # 跳過column欄位項目
    for i, td in enumerate(tr.find_all('td')):
        if len(tr.find_all('td')) == 5:      # 有5個td(欄目) + 第一個月資料
            if i == 0:     # 第一個td是監測項目
                items = td.text
                data.setdefault(items, {})      # {'項目名稱': {}}
            if i == 2:     # 第一個月監測日期
                date = td.text
            if i == 3:     # 第一個監測值
                value = td.text
                data[items][date] = value     # 字典裡面還是字典，進行新增資料
        if len(tr.find_all('td')) == 3:       # 有3個td為第二個月之後資料
            if i == 0:
                date = td.text
            if i == 1:
                value = td.text
                data[items][date] = value
                
# print(data) # 取出所有data
                
import pandas as pd
all_data = pd.DataFrame(data)[:-1]
all_data = pd.DataFrame(data)[:-1]
print(all_data[:8]['SO2'])